# Applied Data Science Capstone - Nathan Anderson

# Segmenting and Clustering Neighborhoods in Toronto 

## Step 1

### Importing Libraries

In [43]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import geocoder

### Scraping the Data from Wiki

In [33]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
canada_data = BeautifulSoup(source, 'lxml')

### Creating a new Dataframe

In [34]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = column_names)
content = canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df = df.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood}, ignore_index=True)
    
df.head()

,PostalCode,Borough,Neighborhood,Postalcode
0,NaN,0,0,0
1,NaN,Not assigned,,M1A
2,NaN,Not assigned,,M2A
3,NaN,North York,Parkwoods,M3A
4,NaN,North York,Victoria Village,M4A


### Cleaning the Dataframe

In [35]:
# Clean Dataframe
df = df[df.Borough != 'Not assigned']
df = df[df.Borough != 0]
df.reset_index(drop = True, inplace = True)

i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
                                 
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()


# pd.set_option("display.max_rows", None, "display.max_columns", None)  # Was used to view the entire data frame.

df.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### The shape of the Dataframe

In [36]:
df.shape

(103, 3)